### 스트림 방식으로 출력하기

In [1]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")
api_key = os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 가져오기

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
llm = ChatOpenAI(model="gpt-4o")

llm.invoke([HumanMessage("잘 지냈어?")])

AIMessage(content='네, 잘 지냈어요! 감사합니다. 당신은 어떻게 지내셨나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 12, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_46bff0e0c8', 'id': 'chatcmpl-C4nANA5Cw8IpC1fFFtD0krjhADIfW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e709cc9e-9d8e-443e-804f-a22bcb3d0f89-0', usage_metadata={'input_tokens': 12, 'output_tokens': 17, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
for c in llm.stream([HumanMessage("잘 지냈어?")]):
    print(c.content, end='|') 

|네|,| 잘| 지|냈|어요|!| 당신|은| 어떻게| 지|내|셨|나요|?||

In [5]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time

In [6]:
# .bind_tools() 메서드를 사용하여 도구를 LLM에 바인딩

# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langchain_core.messages import SystemMessage

messages = [
    SystemMessage(
        content="너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있어."
    ),
    HumanMessage(
        content="서울의 현재 시각을 알려줘."
    )
]

response = llm_with_tools.stream(messages)

# 파편화된 tool_call 청크를 하나로 합치기 
is_first = True
for chunk in response:    
    print("chunk type: ", type(chunk))
    
    if is_first:
        is_first = False
        gathered = chunk
    else:
        gathered += chunk
    
    print("content: ", gathered.content, "tool_call_chunk", gathered.tool_calls)

messages.append(gathered)



chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_QXKh0BWSMt6Foysc8bk0yT8v', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_QXKh0BWSMt6Foysc8bk0yT8v', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_QXKh0BWSMt6Foysc8bk0yT8v', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': ''}, 'id': 'call_QXKh0BWSMt6Foysc8bk0yT8v', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': 'Asia'}, 'id': 'call_QXKh0BWSMt6Foysc8bk0yT8v', 'type': 'tool_c

In [9]:
gathered

AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_QXKh0BWSMt6Foysc8bk0yT8v', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"서울"}', 'name': 'get_current_time'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'service_tier': 'default'}, id='run--2b55548b-4f8f-455b-a237-5891fa4353b6', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '서울'}, 'id': 'call_QXKh0BWSMt6Foysc8bk0yT8v', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"서울"}', 'id': 'call_QXKh0BWSMt6Foysc8bk0yT8v', 'index': 0, 'type': 'tool_call_chunk'}])

In [10]:
for tool_call in gathered.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # tool_dict를 사용하여 도구 이름으로 도구 함수를 선택
    print(tool_call["args"]) # 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '서울'}
Asia/Seoul (서울) 현재시각 2025-08-15 20:50:28 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있어.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='서울의 현재 시각을 알려줘.', additional_kwargs={}, response_metadata={}),
 AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_QXKh0BWSMt6Foysc8bk0yT8v', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"서울"}', 'name': 'get_current_time'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'service_tier': 'default'}, id='run--2b55548b-4f8f-455b-a237-5891fa4353b6', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '서울'}, 'id': 'call_QXKh0BWSMt6Foysc8bk0yT8v', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"서울"}', 'id': 'call_QXKh0BWSMt6Foysc8bk0yT8v', 'index': 0, 'type': 'tool_call_chunk'}]),
 ToolMessage(content='Asia/Se

In [11]:
for c in llm_with_tools.stream(messages):
    print(c.content, end='|')

|서울|의| 현재| 시|각|은| |202|5|년| |8|월| |15|일| |20|시| |50|분| |28|초|입니다|.||